**Задача №6**

**1. Тип приложения:** Веб-приложение. Веб-приложение обеспечивает универсальный доступ из любого устройства с браузером

**2. Основные функции:**
- **Добавление персонажей:** Гендальф может добавить нового персонажа, указав его имя и другую необходимую информацию.
- **Добавление геометки:** Для каждого персонажа можно добавить геометку с датой, временем и координатами его местоположения.
- **Просмотр позиций:** На интерактивной карте отображаются текущие позиции всех персонажей.
- **Просмотр траектории движения:** Гендальф может выбрать персонажа и просмотреть его траекторию движения за определенный период времени.

**3. Дополнительные функции:**
- **Уведомления:** Гендальф может получать уведомления, когда какой-либо из персонажей приближается к определенной локации или покидает ее. Для уведомлений а-ля электронные письма или SMS, потребуется интеграция с соответствующими сервисами.
- **Группировка персонажей:** Возможность группировать персонажей (например, по расам: хоббиты, эльфы, люди и т.д.).
- **История движений:** Возможность просмотреть историю движений конкретного персонажа.
- **Интеграция с книгами и фильмами:** В приложении можно будет посмотреть, где именно находился данный персонаж в определенный момент сюжета книги или фильма. Этого в коде нет, но идея неплохая.

**4. Технологии:**
- **Frontend:** React.js с использованием библиотеки Leaflet для отображения карты.
- **Backend:** Node.js с Express.js для создания серверной части.
- **База данных:** MongoDB для хранения данных о персонажах и их местоположении.

Ниже также команда для установки нужных пакетов:

npm install express mongoose express-basic-auth morgan body-parser

In [ ]:
const
express = require('express');
const
mongoose = require('mongoose');
const
basicAuth = require('express-basic-auth');
const
morgan = require('morgan');
const
bodyParser = require('body-parser');

const
app = express();
const
PORT = process.env.PORT | | 3000;
const
MONGO_URI = process.env.MONGO_URI | | 'mongodb://localhost:27017/lotrTracker';

// Подключение
к
MongoDB
mongoose.connect(MONGO_URI, {useNewUrlParser: true, useUnifiedTopology: true});

// Определение
схем
и
моделей

const
LocationHistorySchema = new
mongoose.Schema({
    type: {type: String, default: 'Point'},
    coordinates: [Number],
    timestamp: Date
});

const
CharacterSchema = new
mongoose.Schema({
                    name: {type: String, unique: true}, // Уникальное
имя
group: String,
locationHistory:[LocationHistorySchema]
});

const
Character = mongoose.model('Character', CharacterSchema);

const
LocationSchema = new
mongoose.Schema({
    name: String,
    coordinates: {
        type: {type: String, default: 'Point'},
        coordinates: [Number]
    }
});

const
Location = mongoose.model('Location', LocationSchema);

const
EventSchema = new
mongoose.Schema({
    title: String,
    description: String,
    characters: [String],
    timestamp: Date
});

const
Event = mongoose.model('Event', EventSchema);

// Middleware
app.use(bodyParser.json());
app.use(morgan('combined'));
app.use(basicAuth({
    users: {'gandalf': 'theonering'},
    challenge: true
}));

// Маршруты

app.post('/addCharacter', (req, res, next) = > {
    const
character = new
Character(req.body);
character.save()
.then(savedCharacter= > res.status(200).send(savedCharacter))
.catch(err= > next(new
AppError('Error adding character', 500)));
});

app.post('/updateLocation', (req, res, next) = > {
    const
{name, coordinates, timestamp} = req.body;

Character.findOne({name})
.then(character= > {
if (!character) throw new AppError('Character not found', 404);

    character.locationHistory.push({coordinates, timestamp});
return character.save();
})
.then(updatedCharacter= > {
return Promise.all([updatedCharacter, Location.find({})]);
})
.then(([updatedCharacter, locations]) = > {
locations.forEach(location= > {
    const
distance = getDistance(updatedCharacter.locationHistory[updatedCharacter.locationHistory.length - 1].coordinates,
                       location.coordinates.coordinates);
if (distance < 1)
{
    console.log(`[Notification]
Character ${updatedCharacter.name} is near ${location.name}
`);
}
});
res.status(200).send(updatedCharacter);
})
.catch(err= > next(err));
});

// Группировка персонажей по группе
app.get('/charactersByGroup/:group', (req, res, next) = > {
Character.find({group: req.params.group})
.then(characters = > res.status(200).send(characters))
.catch(err = > next(new AppError('Error fetching characters by group', 500)));
});

// Обработка ошибок

class AppError extends Error {
constructor(message, statusCode) {
super(message);
this.statusCode = statusCode;
this.status = `${statusCode}`.startsWith('4') ? 'fail': 'error';
this.isOperational = true;
}

}

app.use((err, req, res, next) = > {
    err.statusCode = err.statusCode | | 500;
err.status = err.status | | 'error';

console.error(err.stack);
res.status(err.statusCode).json({
    status: err.status,
    message: err.message
});
});

function
getDistance(coord1, coord2)
{
    const
R = 6371; // радиус
Земли
в
километрах
const
lat1 = coord1[0] * Math.PI / 180;
const
lat2 = coord2[0] * Math.PI / 180;
const
deltaLat = (coord2[0] - coord1[0]) * Math.PI / 180;
const
deltaLon = (coord2[1] - coord1[1]) * Math.PI / 180;

const
a = Math.sin(deltaLat / 2) * Math.sin(deltaLat / 2) + \
    Math.cos(lat1) * Math.cos(lat2) * \
    Math.sin(deltaLon / 2) * Math.sin(deltaLon / 2);

const
c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a));
return R * c; // расстояние
в
километрах
}

app.listen(PORT, () = > {
    console.log(`Server is running
on
http: // localhost:${PORT}
`);
});